In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

data, target = datasets.load_iris(return_X_y=True)
data.shape, target.shape

((150, 4), (150,))

In [2]:
data_train, data_test, target_train, target_test = train_test_split(
    data,
    target,
    test_size=0.3,
    random_state=0
)

In [3]:
clf = svm.SVC(kernel="linear", C=1).fit(data_train, target_train)
clf.score(data_test, target_test)

0.9777777777777777

### Computing cross-validated metrics

In [8]:
from sklearn.model_selection import cross_val_score

clf = svm.SVC(kernel="linear", C=1, random_state=0)
scores = cross_val_score(clf, data, target, cv=5)
len(scores), scores.mean(), scores.std()

(5, 0.9800000000000001, 0.016329931618554516)

In [9]:
from sklearn import metrics

scores = cross_val_score(clf, data, target, cv=5, scoring="f1_macro")
scores

array([0.96658312, 1.        , 0.96658312, 0.96658312, 1.        ])

In [10]:
from sklearn.model_selection import ShuffleSplit

n_sample = data.shape[0]
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(clf, data, target, cv=cv)

array([0.97777778, 0.97777778, 1.        , 0.95555556, 1.        ])

You can also pass a personalized cross_validation function, look at the documentation: https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators

We can also pass a pipeline to the `cross_val_score` or `cross_validate`

In [11]:
from sklearn import pipeline
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
model = pipeline.make_pipeline(scaler, clf)

cross_val_score(model, data, target, cv=5)

array([0.96666667, 1.        , 0.93333333, 0.93333333, 1.        ])

In [12]:
from sklearn import set_config

set_config(display="diagram")

In [13]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=1, kernel='linear', random_state=0))])

`cross_validate` allows:

- Specify multiple metrics for evaluation
- Returns a dict containing fit-times, score-times (and optionally training scores as well as fitted estimators) in adition to the test score

In [15]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score

scoring = ['precision_macro', 'recall_macro']
clf = svm.SVC(kernel="linear", C=1, random_state=0)
scores = cross_validate(clf, data, target, scoring=scoring)

sorted(scores.keys())

['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']

## Cross Validation Iterators

### KFold

In [16]:
import numpy as np
from sklearn.model_selection import KFold

X = ["a", "b", "c", "d"]
kf = KFold(n_splits=2)
for train, test in kf.split(X):
    print(train, test)

[2 3] [0 1]
[0 1] [2 3]


### Repeated K-Fold

In [17]:
from sklearn.model_selection import RepeatedKFold

data = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
rkf = RepeatedKFold(n_splits=2, n_repeats=2, random_state=0)
for train, test in rkf.split(data):
    print(train, test)

[0 1] [2 3]
[2 3] [0 1]
[1 3] [0 2]
[0 2] [1 3]


### LeaveOneOut

In [18]:
from sklearn.model_selection import LeaveOneOut

# is the sample cross validation
X = [1, 2, 3, 4]
loo = LeaveOneOut()
for train, test in loo.split(X):
    print(train, test)

[1 2 3] [0]
[0 2 3] [1]
[0 1 3] [2]
[0 1 2] [3]


In [19]:
from sklearn.model_selection import LeavePOut

lpo = LeavePOut(p=2)
for train, test in lpo.split(X):
    print(train, test)

[2 3] [0 1]
[1 3] [0 2]
[1 2] [0 3]
[0 3] [1 2]
[0 2] [1 3]
[0 1] [2 3]


In [20]:
from sklearn.model_selection import ShuffleSplit

X = np.arange(10)

ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
for train_idx, test_idx in ss.split(X):
    print(train_idx, test_idx)

[9 1 6 7 3 0 5] [2 8 4]
[2 9 8 0 6 7 4] [3 5 1]
[4 5 1 0 6 9 7] [2 3 8]
[2 7 5 8 0 3 4] [6 1 9]
[4 1 0 6 8 9 3] [5 2 7]


Some classification problems can exhibit a large imbalance in the distribution of the target classes: for instance there could be several times more negative samples than positive samples. In such cases it is recommended to use stratified sampling as implemented in **StratifiedKFold** and **StratifiedShuffleSplit** to ensure that relative class frequencies is approximately preserved in each train and validation fold.